In [2]:
using StockFlow
using StockFlow.Syntax

using Catlab
using Catlab.CategoricalAlgebra
using LabelledArrays
using OrdinaryDiffEq
using Plots

using Catlab.Graphics
using Catlab.Programs
using Catlab.Theories
using Catlab.WiringDiagrams


In [3]:
display_uwd(ex) = to_graphviz(ex, box_labels=:name, junction_labels=:variable, edge_attrs=Dict(:len=>"1"));

## Defining the models

In [4]:
#(stock_name=>(inflows, outflows, variables, svariables))
     ## if a stock has no inflow or no outflow, use keyword ":F_NONE"
     ## if a stock has no variables connect to, use keyword ":V_NONE"
     ## if a stock has no sum_variables connect to, use keyword ":SV_NONE"
#(flow=>variable)
#(svariable=>variable)
     ## if sum_variable contributes to no variables, use keywork ":SVV_NONE"

     fexpose_u(u,p,t)=p.β*u.S*u.I/p.N
     finfect_u(u,p,t)=u.E*p.ri
     fdeath_u(u,p,t)=u.I/p.tr * (1.0-p.fH )
     frecover_u(u,p,t)=u.R/p.tw

begin

    SEIRD = StockAndFlowp(
        (:S,:E,:I,:R,:D),
        (
            (:expose_u=>fexpose_u,:S=>:E)=>(:S,:I),
            (:expose_u=>fexpose_u,:S=>:I)=>(:S,:I),
            (:infect_u=>finfect_u,:E=>:I)=>:E,
            (:recover_u=>frecover_u,:I=>:R)=>:I,
            (:death_u=>fdeath_u,:I=>:D)=>:I
        )
    )

end

StockAndFlowp {Flow:5, Stock:5, Link:7, Name:0, FuncFlow:0}
┌──────┬───┬───┬───────────┬────────────┐
│ Flow │ u │ d │     fname │         ϕf │
├──────┼───┼───┼───────────┼────────────┤
│    1 │ 1 │ 2 │  expose_u │  fexpose_u │
│    2 │ 1 │ 3 │  expose_u │  fexpose_u │
│    3 │ 2 │ 3 │  infect_u │  finfect_u │
│    4 │ 3 │ 4 │ recover_u │ frecover_u │
│    5 │ 3 │ 5 │   death_u │   fdeath_u │
└──────┴───┴───┴───────────┴────────────┘
┌───────┬───────┐
│ Stock │ sname │
├───────┼───────┤
│     1 │     S │
│     2 │     E │
│     3 │     I │
│     4 │     R │
│     5 │     D │
└───────┴───────┘
┌──────┬───┬───┐
│ Link │ s │ t │
├──────┼───┼───┤
│    1 │ 1 │ 1 │
│    2 │ 3 │ 1 │
│    3 │ 1 │ 2 │
│    4 │ 3 │ 2 │
│    5 │ 2 │ 3 │
│    6 │ 3 │ 4 │
│    7 │ 3 │ 5 │
└──────┴───┴───┘

In [5]:
Graph(SEIRD)

UndefVarError: UndefVarError: `Graph` not defined

In [6]:
fexpose_v(u,p,t)=p.β*u.S*u.I/p.N
fsvax(u,p,t)=u.E*p.ri

begin

    cross_exp = StockAndFlowp(
        (:S,:I,:V,:Ev),
        (
            (:expose_v=>fexpose_v,:V=>:Ev)=>(:V,:I),
            (:expose_v=>fexpose_v,:V=>:I)=>(:V,:I),
            (:svax=>fsvax,:S=>:V)=>:S
        )
    )

end

StockAndFlowp {Flow:3, Stock:4, Link:5, Name:0, FuncFlow:0}
┌──────┬───┬───┬──────────┬───────────┐
│ Flow │ u │ d │    fname │        ϕf │
├──────┼───┼───┼──────────┼───────────┤
│    1 │ 3 │ 4 │ expose_v │ fexpose_v │
│    2 │ 3 │ 2 │ expose_v │ fexpose_v │
│    3 │ 1 │ 3 │     svax │     fsvax │
└──────┴───┴───┴──────────┴───────────┘
┌───────┬───────┐
│ Stock │ sname │
├───────┼───────┤
│     1 │     S │
│     2 │     I │
│     3 │     V │
│     4 │    Ev │
└───────┴───────┘
┌──────┬───┬───┐
│ Link │ s │ t │
├──────┼───┼───┤
│    1 │ 3 │ 1 │
│    2 │ 2 │ 1 │
│    3 │ 3 │ 2 │
│    4 │ 2 │ 2 │
│    5 │ 1 │ 3 │
└──────┴───┴───┘

In [7]:
frecover_v(u,p,t)=p.β*u.S*u.I/p.N

begin

    VEvR = StockAndFlowp(
        (:Ev,:R,:V),
        (
            (:recover_v=>frecover_v,:Ev=>:R)=>:Ev
        )
    )

end

StockAndFlowp {Flow:1, Stock:3, Link:1, Name:0, FuncFlow:0}
┌──────┬───┬───┬───────────┬────────────┐
│ Flow │ u │ d │     fname │         ϕf │
├──────┼───┼───┼───────────┼────────────┤
│    1 │ 1 │ 2 │ recover_v │ frecover_v │
└──────┴───┴───┴───────────┴────────────┘
┌───────┬───────┐
│ Stock │ sname │
├───────┼───────┤
│     1 │    Ev │
│     2 │     R │
│     3 │     V │
└───────┴───────┘
┌──────┬───┬───┐
│ Link │ s │ t │
├──────┼───┼───┤
│    1 │ 1 │ 1 │
└──────┴───┴───┘

In [21]:
Graph(seirh,"TB")

UndefVarError: UndefVarError: `Graphviz` not defined

### Composition

In [15]:
comp = @relation (S, E, I, R, D, V, Ev) begin
    unvax(S,E,I,R,D)
    vax(V,Ev,R)
    cross(S,I,V,Ev)
end;
display_uwd(comp)

Catlab.Graphics.Graphviz.Graph("G", false, "neato", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Node("n1", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "box1", :label => "unvax")), Catlab.Graphics.Graphviz.Node("n2", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "box2", :label => "vax")), Catlab.Graphics.Graphviz.Node("n3", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "box3", :label => "cross")), Catlab.Graphics.Graphviz.Node("n4", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:height => "0", :id => "outer1", :label => "", :margin => "0", :shape => "none", :style => "invis", :width => "0")), Catlab.Graphics.Graphviz.Node("n5", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:height => "0", :id => "outer2", :label => "", :margin => "0", :shape => "none", :sty

In [17]:
# Open three Stock and Flow Diagrams
openseird = Open(SEIRD, [:S], [:E], [:I], [:R],[:D])
openv = Open(VEvR, [:V], [:Ev], [:R])
opencross = Open(cross_exp, [:S], [:I],[:V],[:Ev])
# Compose those three models according the UWD-algebra
openall = oapply(comp, [openseird, openv, opencross])
# Generate the composed model (Stock and Flow Diagram)
mod = apex(openall)

StockAndFlowp {Flow:9, Stock:7, Link:13, Name:0, FuncFlow:0}
┌──────┬───┬───┬───────────┬────────────┐
│ Flow │ u │ d │     fname │         ϕf │
├──────┼───┼───┼───────────┼────────────┤
│    1 │ 1 │ 2 │  expose_u │  fexpose_u │
│    2 │ 1 │ 3 │  expose_u │  fexpose_u │
│    3 │ 2 │ 3 │  infect_u │  finfect_u │
│    4 │ 3 │ 4 │ recover_u │ frecover_u │
│    5 │ 3 │ 5 │   death_u │   fdeath_u │
│    6 │ 6 │ 4 │ recover_v │ frecover_v │
│    7 │ 7 │ 6 │  expose_v │  fexpose_v │
│    8 │ 7 │ 3 │  expose_v │  fexpose_v │
│    9 │ 1 │ 7 │      svax │      fsvax │
└──────┴───┴───┴───────────┴────────────┘
┌───────┬───────┐
│ Stock │ sname │
├───────┼───────┤
│     1 │     S │
│     2 │     E │
│     3 │     I │
│     4 │     R │
│     5 │     D │
│     6 │    Ev │
│     7 │     V │
└───────┴───────┘
┌──────┬───┬───┐
│ Link │ s │ t │
├──────┼───┼───┤
│    1 │ 1 │ 1 │
│    2 │ 3 │ 1 │
│    3 │ 1 │ 2 │
│    4 │ 3 │ 2 │
│    5 │ 2 │ 3 │
│    6 │ 3 │ 4 │
│    7 │ 3 │ 5 │
│    8 │ 6 │ 6 │
│    9 │ 7 │ 7 │
│   10 │ 3 │ 7 │
│   11 │ 7 │ 8 │
│   12 │ 3 │ 8 │
│   13 │ 1 │ 9 │
└──────┴───┴───┘

In [20]:
StockFlow.Graph(mod)

UndefVarError: UndefVarError: `Graphviz` not defined

### Stratification